In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

C:\Users\UTEA\anaconda3\envs\informes_utea\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# ruta de carpera donde estan las llaves
ruta = open('../../ruta.txt')
ruta = ruta.readlines()[0]

In [4]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [5]:
semana = 14

In [19]:
query_semana = {'query': 'select id, canhero from dataset_315614 where date_part(\'week\', fecha_de_registro)={sem} order by id'.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
select_semana = amigocloud.get(project, query_semana)
data_semana = select_semana['data']

In [20]:
data_semana

[{'id': 18, 'canhero': '4171 / FRITZER CUELLAR SAMIR EDUARDO '},
 {'id': 19, 'canhero': '633 / AGROP. VICENTE ROCA GIL SRL '},
 {'id': 20, 'canhero': '633 / AGROP. VICENTE ROCA GIL SRL '},
 {'id': 21, 'canhero': '3421 / ESPINOZA HURTADO ARMANDO'},
 {'id': 22, 'canhero': '3421 / ESPINOZA HURTADO ARMANDO'},
 {'id': 23, 'canhero': '86 / AGUILERA TARADELLES JOSE LUIS '},
 {'id': 24, 'canhero': '86 / AGUILERA TARADELLES JOSE LUIS '},
 {'id': 25, 'canhero': '3415 / ESPINOZA VILLEGAS GERARDO '},
 {'id': 26, 'canhero': '3378 / ESCOBAR MORATO VICTOR HUGO'},
 {'id': 27, 'canhero': '3378 / ESCOBAR MORATO VICTOR HUGO'},
 {'id': 28, 'canhero': '854 / AGROPECUARIA LOPEZ QUIROZ S.R.L.'}]

In [28]:
for id in data_semana:
    print('ID inspeccion: ', id['id'])
    id_insp = id['id']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]

    # convertir fecha a formato dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

    # los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
    # se modifica el dict para agregar 'k_' al inicio de todas llaves
    data = {'k_'+key : value for (key, value) in data.items()}
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # FOTOS
    query_fotos = {'query': 'select gal.s3_filename, fot.obs from dataset_315689 fot\
                                inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                                where fot.insp_ref_fot=\'{id}\''.format(id=insp.k_amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # CREAR REPORTE
    doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras_preventivo.docx')
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(50)),
            'obs': foto['obs']
        })
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    #context = {'insp':insp}
    doc.render(context)
    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.k_canhero.split(' / ')
    file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)
    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(ruta + r'/informes/_' + file_name + '.docx')
    print(file_name)
print('-------------------FIN')

ID inspeccion:  18
4171_ECP_04-04-2023_FRITZER CUELLAR SAMIR EDUARDO _18
ID inspeccion:  19
633_ECP_04-04-2023_AGROP. VICENTE ROCA GIL SRL _19
ID inspeccion:  20
633_ECP_04-04-2023_AGROP. VICENTE ROCA GIL SRL _20
ID inspeccion:  21
3421_ECP_04-04-2023_ESPINOZA HURTADO ARMANDO_21
ID inspeccion:  22
3421_ECP_04-04-2023_ESPINOZA HURTADO ARMANDO_22
ID inspeccion:  23
86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _23
ID inspeccion:  24
86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _24
ID inspeccion:  25
3415_ECP_05-04-2023_ESPINOZA VILLEGAS GERARDO _25
ID inspeccion:  26
3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_26
ID inspeccion:  27
3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_27
ID inspeccion:  28
854_ECP_06-04-2023_AGROPECUARIA LOPEZ QUIROZ S.R.L._28
-------------------FIN


## Convertir a PDF

In [19]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 11 INFORMES
11 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_26.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_27.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3415_ECP_05-04-2023_ESPINOZA VILLEGAS GERARDO _25.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3421_ECP_04-04-2023_ESPINOZA HURTADO ARMANDO_21.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3421_ECP_04-04-2023_ESPINOZA HURTADO ARMANDO_22.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/4171_ECP_04-04-2023_FRITZER CUELLAR SAMIR EDUARDO _18.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/633_ECP_04-04-2023_AGROP. VICENTE ROCA GIL SRL _19.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/633_ECP_04-04-2023_AGROP. VICENT

In [25]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_26.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_26.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_26.pdf




  0%|                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.22s/it]


CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_27.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_27.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3378_ECP_06-04-2023_ESCOBAR MORATO VICTOR HUGO_27.pdf




  0%|                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.23s/it]


CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3415_ECP_05-04-2023_ESPINOZA VILLEGAS GERARDO _25.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3415_ECP_05-04-2023_ESPINOZA VILLEGAS GERARDO _25.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3415_ECP_05-04-2023_ESPINOZA VILLEGAS GERARDO _25.pdf


...

CONVIRTIENDO... INFORME 10 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _23.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _23.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _23.pdf




  0%|                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.38s/it]


CONVIRTIENDO... INFORME 11 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _24.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _24.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/86_ECP_04-04-2023_AGUILERA TARADELLES JOSE LUIS _24.pdf




  0%|                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.56s/it]

END
